In [1]:
import lightgbm as lgb
import glob
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
def calcualte_acc(x, y):
    count = 0
    for i, j in enumerate(x):
        if j == y[i]:
            count += 1
    print(count/len(x))

# beauty

In [4]:
fashion_paths = glob.glob('predictions/fashion/*')

In [5]:
embed_paths = glob.glob('predictions/fashion_embed/*')

In [6]:
fashion_paths += embed_paths

In [7]:
fashion_paths

['predictions/fashion/gru',
 'predictions/fashion/conv1d_1',
 'predictions/fashion/lstm',
 'predictions/fashion/conv1d_2',
 'predictions/fashion/conv1d_3',
 'predictions/fashion/conv1d_2layers']

In [8]:
test_paths = glob.glob('predictions/test/*')

In [13]:
test_paths = ['predictions/test/gru.npy',
 'predictions/test/conv1d_1.npy',
 'predictions/test/lstm.npy',
 'predictions/test/conv1d_2.npy',
 'predictions/test/conv1d_3.npy',
 'predictions/test/con1d_2layers.npy']

In [14]:
test_set = np.load(test_paths[0])[:, 17:31]
print(test_set.shape)
for i in test_paths[1:]:
    test_set = np.append(test_set, np.load(i)[:, 17:31], axis=1)

(172402, 14)


In [15]:
test_set = test_set[76545:131985, :]

In [16]:
num_classes = 14

In [17]:
params={
        "objective" : "multiclass",
        "num_class" : num_classes,
        "num_leaves" : 63,
        "max_depth": -1,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.9,  # subsample
        "feature_fraction" : 0.9,  # colsample_bytree
        "bagging_freq" : 5,        # subsample_freq
        "bagging_seed" : 2019,
        'max_bin': 63,
        #'device': 'gpu',
        'metric': 'multi_logloss',
        'multi_logloss': 'gbdt'
}

In [18]:
indexes = np.load('np_array/trn_index_f_fold.npy')
for i in indexes:
    print(i.shape)

(43941,)
(43941,)
(43940,)
(43940,)
(43940,)


In [19]:
for i in range(5):
    print(glob.glob(fashion_paths[0]+'/*')[i])
    print(np.load(glob.glob(fashion_paths[0]+'/*')[i]).shape)

predictions/fashion/gru/fashion_predictions_3.npy
(43940, 14)
predictions/fashion/gru/fashion_predictions_1.npy
(43941, 14)
predictions/fashion/gru/fashion_predictions_0.npy
(43941, 14)
predictions/fashion/gru/fashion_predictions_4.npy
(43940, 14)
predictions/fashion/gru/fashion_predictions_2.npy
(43940, 14)


In [20]:
test_output = np.zeros((test_set.shape[0], 14))
for i in range(5):
    print('fold_{}'.format(i))
    lst = [0,1,2,3,4]
    lst.remove(i)

    trn_set = np.load(glob.glob(fashion_paths[0]+'/*')[lst[0]])
    val_set = np.load(glob.glob(fashion_paths[0]+'/*')[i])
    
    for j in fashion_paths[1:]:
        trn_set = np.append(trn_set, np.load(glob.glob(j+'/*')[lst[0]]), axis=1)
        val_set = np.append(val_set, np.load(glob.glob(j+'/*')[i]), axis=1)
    #trn_set = trn_set.reshape(trn_set.shape[0], 1)
        
    for x in lst[1:]:
        one_row = np.load(glob.glob(fashion_paths[0]+'/*')[x])
        for j in fashion_paths[1:]:
            one_row = np.append(one_row, np.load(glob.glob(j+'/*')[x]), axis=1)
        trn_set = np.append(trn_set, one_row, axis=0)
    
    indexes = np.load('np_array/trn_index_f_fold.npy')
    index = int(glob.glob(fashion_paths[0] + '/*')[i][-5])
    print(index)
    val_index = indexes[index]
    other_lst = [0,1,2,3,4]
    other_lst.remove(index)
    trn_index = indexes[other_lst[0]]
    other_lst.remove(other_lst[0])
    for num in other_lst:
        trn_index = np.append(trn_index, indexes[num])
        
    labels_trn = train.iloc[trn_index, 2]
    labels_trn = np.array(labels_trn) - 17

    labels_val = train.iloc[val_index, 2]
    labels_val = np.array(labels_val) - 17
    
    print(trn_set.shape)
    print(val_set.shape)
    lgtrain, lgval = lgb.Dataset(trn_set, labels_trn), lgb.Dataset(val_set, labels_val)
    lgbmodel = lgb.train(params, lgtrain, 2000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100)
    pred = lgbmodel.predict(val_set)
    np.save('first_layer/five_nlp_{}'.format(index), pred)
    pred = lgbmodel.predict(test_set)
    test_output += pred/5

fold_0
3
(175762, 84)
(43940, 84)
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_logloss: 1.32262	valid_1's multi_logloss: 1.37975
[200]	training's multi_logloss: 1.0869	valid_1's multi_logloss: 1.17557
[300]	training's multi_logloss: 0.983217	valid_1's multi_logloss: 1.10094
[400]	training's multi_logloss: 0.92455	valid_1's multi_logloss: 1.0702
[500]	training's multi_logloss: 0.884141	valid_1's multi_logloss: 1.05631
[600]	training's multi_logloss: 0.85274	valid_1's multi_logloss: 1.04972
[700]	training's multi_logloss: 0.826359	valid_1's multi_logloss: 1.0465
[800]	training's multi_logloss: 0.803476	valid_1's multi_logloss: 1.0452
[900]	training's multi_logloss: 0.78312	valid_1's multi_logloss: 1.04461
[1000]	training's multi_logloss: 0.764746	valid_1's multi_logloss: 1.04454
[1100]	training's multi_logloss: 0.747651	valid_1's multi_logloss: 1.04472
Early stopping, best iteration is:
[1000]	training's multi_logloss: 0.764746	valid_1's multi_log

In [21]:
np.save('fashion_five_test', test_output)

In [40]:
predictions = []
for i in pred:
    predictions.append(list(i).index(i.max()))

In [41]:
labels_val.shape

(43940,)

In [42]:
len(predictions)

43940

In [43]:
calcualte_acc(predictions, labels_val)

0.6520937642239417
